In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open County data

In [24]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)

## Get Central American Data

In [25]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/groups/B03001.json")
columns_obj = r.json()

### Get columns to query and rename for later

In [26]:
columns = []
rename_vars = {}
variables = columns_obj["variables"]
for name, variable in list(variables.items()):
    v_split = variable["label"].split("!!")
    if len(v_split) < 3:
        continue

    if v_split[0] == "Estimate":
        label = v_split[-1]
        rename_vars[name] = label

    if (name.endswith("E") or name.endswith("M")) and v_split[-2] == "South American:":
        columns.append(name)

In [27]:
columns.append("GEO_ID")
columns_formatted = ",".join(columns)

In [28]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5?get={columns_formatted}&for=state:*"
)

In [29]:
data = response.json()
columns = data[0]
rows = data[1:]
sa_df = pd.DataFrame(rows, columns=columns)

In [30]:
estimate_cols = [col for col in sa_df.columns if col.endswith("E")]

In [31]:
sa_formtted_df = sa_df[["GEO_ID", *estimate_cols]]
sa_formtted_df[estimate_cols] = sa_formtted_df[estimate_cols].astype(int)

/tmp/ipykernel_66713/2903963453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_formtted_df[estimate_cols] = sa_formtted_df[estimate_cols].astype(int)


In [32]:
sa_formtted_df["most_common_ancestry_raw"] = sa_formtted_df[estimate_cols].idxmax(
    axis=1
)

/tmp/ipykernel_66713/2456611500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_formtted_df["most_common_ancestry_raw"] = sa_formtted_df[estimate_cols].idxmax(


In [33]:
def check_margin_error(row) -> str:
    geo_id = row["GEO_ID"]
    ethnicity_col = row["most_common_ancestry_raw"]
    val = row[ethnicity_col]

    if not val:
        return None

    moe_col = ethnicity_col.replace("E", "M")
    moe_val = int(sa_df[sa_df["GEO_ID"] == geo_id][moe_col])

    rmoe_val = abs(moe_val / val)
    if rmoe_val < 0.50:
        return variables[ethnicity_col]["label"].split("!!")[-1]
    else:
        return None

In [34]:
sa_formtted_df["most_common_ancestry"] = sa_formtted_df.apply(
    lambda row: check_margin_error(row), axis=1
)

/tmp/ipykernel_66713/1707469957.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  moe_val = int(sa_df[sa_df["GEO_ID"] == geo_id][moe_col])
/tmp/ipykernel_66713/342132343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_formtted_df["most_common_ancestry"] = sa_formtted_df.apply(


In [39]:
rename_vars["GEO_ID"] = "AFFGEOID"
sa_formtted_df = sa_formtted_df.rename(columns=rename_vars)

## Merge Data

In [42]:
sa_gdf = states_gdf.merge(sa_formtted_df, on="AFFGEOID", how="inner")

In [43]:
sa_gdf = sa_gdf.to_crs(9311)
sa_gdf.to_file("data/South_American_Origin_Per_State.gpkg")

In [44]:
sa_gdf.groupby("most_common_ancestry").size().reset_index(name="COUNT").sort_values(
    "COUNT", ascending=False
)

,most_common_ancestry,COUNT
0,Colombian,35
2,Peruvian,7
1,Ecuadorian,5
3,Venezuelan,5
